In [ ]:
# Universal Setup for Backend Environment
import sys
import os
import subprocess
from pathlib import Path

def setup_environment():
    """Setup the environment by installing necessary dependencies and setting paths."""
    # Get the backend directory. If we are in 'backend', it is cwd.
    backend_dir = Path.cwd()
    if backend_dir.name != 'backend':
        # Search for backend
        if (backend_dir / 'backend').exists():
             backend_dir = backend_dir / 'backend'
        elif (backend_dir.parent / 'backend').exists():
             backend_dir = backend_dir.parent / 'backend'
    
    # Add src to path if it exists (for 'from agent import ...' style)
    src_dir = backend_dir / 'src'
    if src_dir.exists():
        if str(src_dir) not in sys.path:
            sys.path.append(str(src_dir))
            print(f"✅ Added {src_dir} to sys.path")
    
    if str(backend_dir) not in sys.path:
        sys.path.append(str(backend_dir))
        
    # Verify backend/agent can be imported
    try:
        import agent
        print("✅ Agent module found and imported.")
    except ImportError:
        print("❌ Agent module not found. Installing dependencies...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-e", str(backend_dir)])
        print("✅ Backend installed in editable mode.")

setup_environment()

In [ ]:
# --- COLAB SETUP START ---
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    # 1. Clone the repository
    !git clone https://github.com/google-gemini/gemini-fullstack-langgraph-quickstart
    %cd gemini-fullstack-langgraph-quickstart/backend

    # 2. Prepare Environment (Resolving Conflicts)
    import sys
    print("Uninstalling conflicting pre-installed packages...")
    !pip uninstall -y google-ai-generativelanguage tensorflow grpcio-status

    # Pre-install PyTorch Nightly if Python 3.13+ is detected
    if sys.version_info >= (3, 13):
        print("Detected Python 3.13+. Installing PyTorch Nightly...")
        !pip install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cpu

    # 3. Install Backend
    !pip install .

    # 4. Set API Key
    import os
    from google.colab import userdata
    try:
        os.environ["GEMINI_API_KEY"] = userdata.get('GEMINI_API_KEY')
    except:
        print("Please enter your Gemini API Key:")
        os.environ["GEMINI_API_KEY"] = input()
# --- COLAB SETUP END ---

In [ ]:
# --- MODEL CONFIGURATION ---
# @title Select Gemini Model
# @markdown Choose the Gemini model to use. Only Gemini 2.5 models are currently accessible via the API.

MODEL_STRATEGY = "Gemini 2.5 Flash (Recommended)" # @param ["Gemini 2.5 Flash (Recommended)", "Gemini 2.5 Flash-Lite (Fastest)", "Gemini 2.5 Pro (Best Quality)"]

import os

# Map selection to model ID
# Note: Gemini 1.5 and 2.0 models are deprecated/not accessible via this API
if MODEL_STRATEGY == "Gemini 2.5 Flash (Recommended)":
    SELECTED_MODEL = "gemini-2.5-flash"
elif MODEL_STRATEGY == "Gemini 2.5 Flash-Lite (Fastest)":
    SELECTED_MODEL = "gemini-2.5-flash-lite"
elif MODEL_STRATEGY == "Gemini 2.5 Pro (Best Quality)":
    SELECTED_MODEL = "gemini-2.5-pro"
else:
    # Default fallback
    SELECTED_MODEL = "gemini-2.5-flash"

print(f"Selected Model: {SELECTED_MODEL}")
print(f"Strategy: {MODEL_STRATEGY}")

# Set Environment Variables to override defaults
os.environ["QUERY_GENERATOR_MODEL"] = SELECTED_MODEL
os.environ["REFLECTION_MODEL"] = SELECTED_MODEL
os.environ["ANSWER_MODEL"] = SELECTED_MODEL
os.environ["TOOLS_MODEL"] = SELECTED_MODEL

In [ ]:
# --- MODEL VERIFICATION (Optional) ---
# @title Verify Model Configuration
# @markdown Run this cell to verify that your API key is configured correctly and the selected model is available.

import os

# Check if API key is set
if "GEMINI_API_KEY" not in os.environ:
    print("⚠️  GEMINI_API_KEY not found in environment variables!")
    print("   Please set it before proceeding:")
    print("   export GEMINI_API_KEY='your-api-key-here'")
else:
    try:
        from google import genai
        
        # Initialize the client
        client = genai.Client(api_key=os.environ["GEMINI_API_KEY"])
        
        # Test the selected model
        print(f"🧪 Testing model: {SELECTED_MODEL}")
        response = client.models.generate_content(
            model=SELECTED_MODEL,
            contents="Explain how AI works in a few words"
        )
        
        print(f"✅ Model verification successful!")
        print(f"   Model: {SELECTED_MODEL}")
        print(f"   Response: {response.text[:100]}...")
        
    except ImportError:
        print("⚠️  google-genai package not installed!")
        print("   Installing now...")
        import subprocess
        import sys
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "google-genai"])
        print("✅ Installed! Please re-run this cell.")
        
    except Exception as e:
        print(f"❌ Model verification failed: {e}")
        print(f"   This could mean:")
        print(f"   - Invalid API key")
        print(f"   - Model '{SELECTED_MODEL}' not available in your region")
        print(f"   - Quota/billing issues (for experimental models)")
        print(f"   - Network connectivity issues")

# Deep Research Agent: Core Workflow

This notebook demonstrates the core research workflow of the Deep Research Agent, including:
1. Query Decomposition
2. Web Search & RAG Ingestion
3. Evidence Auditing & Verification
4. Final Synthesis
5. Artifact Export

## 1. Setup & Dependencies

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import os

# Add backend/src to path so we can import modules
sys.path.append(os.path.abspath("../backend/src"))

In [ ]:
from agent.deep_search_agent import DeepSearchAgent
from langchain_google_genai import ChatGoogleGenerativeAI
from agent.configuration import Configuration
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

## 2. Initialize Agent Components

In [ ]:
# Initialize LLM
# We use the SELECTED_MODEL from the config cell above
model_name = os.environ.get("ANSWER_MODEL", "gemini-1.5-flash")

try:
    print(f"Initializing LLM with model: {model_name}")
    llm = ChatGoogleGenerativeAI(model=model_name, temperature=0)
except Exception as e:
    print(f"Error initializing LLM: {e}")
    # Fallback for testing
    llm = None

if llm:
    # Initialize Agent
    # Note: DeepSearchAgent internally uses the Configuration object which reads from env vars
    agent = DeepSearchAgent(llm_client=llm)
    print("Agent initialized successfully.")
else:
    print("Failed to initialize Agent due to LLM error.")

## 3. Single Query Research Demo

In [ ]:
query = "Latest advances in renewable energy storage"
print(f"Starting research on: {query}")

if agent:
    final_answer = agent.research(query)
    print("\n=== FINAL ANSWER ===\n")
    print(final_answer)
else:
    print("Agent not initialized.")

## 4. RAG Verification Deep Dive

Inspect the internal state of the RAG system to verify ingestion and retrieval.

In [ ]:
if agent:
    # Inspect Doc Store
    print(f"Total documents indexed: {len(agent.rag.doc_store)}")

    # Sample a document
    if len(agent.rag.doc_store) > 0:
        doc_id = list(agent.rag.doc_store.keys())[0]
        print(f"\nSample Document ({doc_id}):")
        print(agent.rag.doc_store[doc_id])

In [ ]:
if agent:
    # Test Retrieval
    test_query = "battery cost reduction"
    results = agent.rag.retrieve(test_query, top_k=3)

    print(f"\nRetrieval results for '{test_query}':")
    for doc, score in results:
        print(f"- [{score:.2f}] {doc.content[:100]}...")

## 5. Export Research Artifacts

In [ ]:
if agent:
    state = agent.rag.export_state()
    print("RAG State Export:", state)

    # Retrieve all docs for export
    all_docs = agent.get_retrieved_documents()
    print(f"\nPrepared {len(all_docs)} documents for export.")